<a href="https://colab.research.google.com/github/jocelynbaduria/cmpe-297_SOTA/blob/BERT_Pretrained/Transformer_MLM_and__NSP_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ReadMe :
1. Install some module gradio app and weights and biases for experiment tracking.

2. Reproduce the code for understanding of Transformer MLM and NSP.

Reference : https://github.com/jamescalam/transformers/blob/main/course/training/08_mlm_and_nsp_training.ipynb



In [1]:
!pip install gradio -q
# Install wandb for experiment tracking
!pip install --upgrade https://github.com/wandb/client/archive/feature/code-save.zip -q
!pip install transformers -q

     |████████████████████████████████| 3.6 MB 4.3 MB/s 
     |████████████████████████████████| 1.9 MB 52.6 MB/s 
     |████████████████████████████████| 206 kB 53.4 MB/s 
     |████████████████████████████████| 3.5 MB 37.3 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
     |████████████████████████████████| 961 kB 46.0 MB/s 
     \ 10.4 MB 1.3 MB/s
     |████████████████████████████████| 180 kB 2.7 MB/s 
     |████████████████████████████████| 76 kB 4.4 MB/s 
     |████████████████████████████████| 139 kB 30.6 MB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 70 kB 8.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 2.9 MB 4.1 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 58.1 MB/s 
     |████████████████████████████████| 3.3 MB 28.5 MB/s 
     |████████████████████████████████| 596 kB 46.0 MB/s 


In [2]:
from transformers import BertTokenizer, BertForPreTraining
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')

with open('/content/drive/MyDrive/cmpe-297-SOTA/data/text/meditations/clean.txt', 'r') as fp:
    text = fp.read().split('\n')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Other imports 
import wandb
wandb.login()
from wandb.keras import WandbCallback
from tqdm import tqdm

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
text[:3]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.']

Modify data into mix of non-random sentences, and random sentences for NSP.
Creates a 'bag of random sentences that we can pull from when selecting a random sentence B.

In [5]:
bag = [item for sentence in text for item in sentence.split('.') if item != '']
bag_size = len(bag)

Create a 50/50 NSP training data

In [6]:
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    sentences = [ sentence for sentence in paragraph.split('.') if sentence != '']

    num_sentences = len(sentences)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 wether isNextSentence or Not NextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence 
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this no NotNetSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            label.append(1)

Tokenize the data using a truncation/padding max_length of 512

In [7]:
# inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length') - out of memeory
# inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=None, truncation=True, padding='max_length') - works 
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=256, truncation=True, padding='max_length')

In [8]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

Create a labels tensor for MLM, and the next_sentence_label for NSP. The next_sentence_label tensor is simply a LongTensor([label]).T 

In [9]:
inputs['next_sentence_label'] = torch.LongTensor([label]).T

Labels tensors is simply a clone of the input_ids tensor before masking

In [10]:
inputs['labels'] = inputs.input_ids.detach().clone()

Now we mask tokens in the input_ids tensor using the 15% probability for MLM - ensuring we don't mask CLS, SEP, or PAD tokens.

In [11]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)

In [12]:
# create mask array
mask_array = (rand < 0.15) * (inputs.input_ids != 101) * \
              (inputs.input_ids != 102) * (inputs.input_ids != 0)

Take the indices of each True value within each vector

In [13]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_array[i].nonzero()).tolist()
    )

Apply these indices to each row in input_ids, assigning each value at these indices a value of 103.

In [14]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [15]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label', 'labels'])

Now all inputs and labels are ready, so we can begin setting up inputs to be fed into model during training. Create a PyTorch dataset from our data.

In [16]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

Initialize the data using the MeditationsDataset class


In [17]:
dataset = MeditationsDataset(inputs)

Initialize the Dataloader to load the data into model training


In [18]:
# loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

Setup the training loop setting and the GPU/CPU usage

In [19]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move the model to the selected device
model.to(device)

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [20]:
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))

Torch 1.9.0+cu111 CUDA 11.1
Device: cuda:0


Activate the training mode of our model, and initialize the optimizer (Adam with weighted decay - reduces chance of overfitting).

In [21]:
# Initialize the weights and biases
wandb.init(project="mlm_nsp", id="bert_pretrained_batchsize1_maxlength256")

from transformers import AdamW
# activate training mode
model.train()

# intialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

wandb: Wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [22]:
!sudo fuser -v /dev/nvidia*

                     USER        PID ACCESS COMMAND
/dev/nvidia0:        root         78 F...m python3
/dev/nvidiactl:      root         78 F...m python3
/dev/nvidia-uvm:     root         78 F.... python3


In [23]:
!sudo kill -9 PID

kill: failed to parse argument: 'PID'


Do the training loop for two epochs(you can change epochs for modificatiions)

In [24]:
from tqdm.notebook import tqdm # for progress bar

epochs = 2

for epoch in range(epochs):
  # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        next_sentence_label = batch['next_sentence_label'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/317 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


  0%|          | 0/317 [00:00<?, ?it/s]

Add the gradio app for testing


In [25]:
import gradio as gr
from transformers import BertTokenizer, BertForPreTraining
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')

def generate_text(inputs):
    input_ids = tokenizer.encode(inputs, return_tensors='pt')
    beam_output = model.generate(input_ids, max_length=100, num_beams=5,
          no_repeat_ngram_size=2, early_stopping=True)
    output = tokenizer.decode(beam_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return ".".join(output.split(".")[:-1]) + "."

output_text = gr.outputs.Textbox()
gr.Interface(generate_text,"textbox", output_text, title="Meditation", description="From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.").launch()

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://23399.gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://23399.gradio.app')